In [ ]:
import pandas as pd
import numpy as np
import sqlite3
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from collections import defaultdict
import mplhep as hep
hep.style.use('CMS')

import warnings
warnings.filterwarnings("ignore")

In [ ]:
import os, sys

path2add = os.path.normpath(os.path.abspath(os.path.join(os.path.curdir, os.path.pardir, 'TestBeam')))
print(path2add)

if (not (path2add in sys.path)) :
    sys.path.append(path2add)

from beamtest_analysis_helper import plot_BL_and_NW

## Load Run Info

In [ ]:
run_time_df = pd.read_csv('./tb_run_info/DESY_Jun2024_TB_run_info.csv')
run_time_df.dropna(inplace=True)

run_time_df['Start_Time'] = run_time_df['Start_Time'] + ", 2024"
run_time_df['Start_Time'] = pd.to_datetime(run_time_df['Start_Time'])

run_time_df['Run'] = run_time_df['Run'].astype('uint8')
run_time_df = run_time_df.astype({col: 'uint16' for col in run_time_df.columns if col.startswith('HV')})
run_time_df = run_time_df.astype({col: 'uint8' for col in run_time_df.columns if col.startswith('offset')})

In [ ]:
# Remove "hour" from Duration column
run_time_df['Duration'] = run_time_df['Duration'].str.replace('hour', '').str.strip()

# Directly add the duration to the Start_Time
run_time_df['Duration'] = run_time_df['Duration'].astype(float)
run_time_df['End_Time'] = run_time_df['Start_Time'] + pd.to_timedelta(run_time_df['Duration'], unit='h')

# Remove when Duration = 0
run_time_df = run_time_df.loc[run_time_df['Duration'] != 0]

In [ ]:
run_time_df

## Baseline plots

In [ ]:
con = sqlite3.connect('/media/daq/X9/DESYJune2024/ETROC-History/BaselineHistory.sqlite')

baseline_df = pd.read_sql_query("SELECT * from baselines", con)

baseline_df['timestamp'] = pd.to_datetime(baseline_df['timestamp'], format='%Y-%m-%d %H:%M:%S.%f')
baseline_df['timestamp'] = baseline_df['timestamp'].dt.strftime('%Y-%m-%d %H:%M:%S')
baseline_df['timestamp'] = pd.to_datetime(baseline_df['timestamp'])

In [ ]:
### Add board names in the dictionary

config_dict = {
    'ET2p01_Bar4': {
        'plot_title': '(FBK UFSD4 W15) BAR_4',
        'chip_type': 'T',
        'channel': 0,
    },
    'ET2p01_Bar12': {
        'plot_title': 'BAR_12',
        'chip_type': 'T',
        'channel': 1,
    },
    'ET2p01_Bar13': {
        'plot_title': 'ET2p01_Bar13',
        'chip_type': 'T',
        'channel': 2,
    },
    'ET2p01_CNM13HPK': {
        'plot_title': '(HPK Split-3 W16 P6) BB_1-3',
        'chip_type': 'F',
        'channel': 3,
    },
}

### Make BL and NW map

In [ ]:
plot_BL_and_NW(run_time_df=run_time_df, which_run=35, baseline_df=baseline_df, config_dict=config_dict, which_val='baseline')

In [ ]:
plot_BL_and_NW(run_time_df=run_time_df, which_run=35, baseline_df=baseline_df, config_dict=config_dict, which_val='noise_width')

### Select data

In [ ]:
# selected_baselines = defaultdict(pd.DataFrame)

# for index, row in run_time_df.iterrows():
#     cut_start_time = row['Start_Time'] - pd.Timedelta(minutes=30)
#     cut_end_time = row['End_Time'] + pd.Timedelta(minutes=10)

#     filtered_df = interest_df.loc[(interest_df['timestamp'] > cut_start_time) & (interest_df['timestamp'] < cut_end_time)]
#     selected_baselines[row['Run']] = filtered_df

In [ ]:
# selected_baselines.keys()

### Baseline over time

In [ ]:
fig, ax = plt.subplots(figsize=(20, 10))
hep.cms.text(loc=0, ax=ax, text="ETL ETROC Test Beam", fontsize=18)
# ax.set_title('Baseline of pixel (8, 11)', size=18, loc="right")

date_form = mdates.DateFormatter("%m-%d\n%H:%M")
ax.xaxis.set_major_formatter(date_form)

cut_time = pd.to_datetime('2024-05-01 22:00:00')

interest_df = baseline_df.loc[(baseline_df['row'] == 8) & (baseline_df['col'] == 11)]

for iboard in interest_df['chip_name'].unique():
    tmp_df = interest_df.loc[interest_df['chip_name']==iboard]
    tmp_df = tmp_df.loc[tmp_df['timestamp'] > cut_time]
    ax.plot(tmp_df['timestamp'], tmp_df['baseline'], label=f'{iboard}', marker='.')

text_y = [200, 370]
draw_dates = True

if draw_dates:
    for index, row in run_time_df.iterrows():

        if index%2 == 0:
            yval = text_y[0]
        else:
            yval = text_y[1]

        plt.axvline(x=row['Start_Time'], color='magenta', linestyle='dashed')
        ax.text(x=(row['Start_Time'] - pd.Timedelta(hours=1.2)), y=yval, s=f"Run {row['Run']} Start", rotation=90, va='center', fontsize=12, color='magenta')
        plt.axvline(x=row['End_Time'], color='black', linestyle='dashdot')
        ax.text(x=(row['End_Time'] + pd.Timedelta(minutes=15)), y=yval, s=f"Run {row['Run']} End", rotation=90, va='center', fontsize=12)

ax.legend(loc='center left', fontsize=16, title='Pixel (8, 11)', title_fontsize=18)

In [ ]:
pixel_plot_list_row,pixel_plot_list_col = np.meshgrid([2, 5, 7, 8, 9, 13], [2, 5, 7, 8, 9, 13])
pixel_plot_list = list(zip(pixel_plot_list_row.flatten(),pixel_plot_list_col.flatten()))

draw_dates = True
cut_time = pd.to_datetime('2024-05-01 22:00:00')

for iboard in interest_df['chip_name'].unique():
    fig, ax = plt.subplots(figsize=(20, 10))
    hep.cms.text(loc=0, ax=ax, text="Phase-2 Preliminary", fontsize=25)
    ax.set_title(f'Baseline of board {iboard}', size=17, loc="right")
    date_form = mdates.DateFormatter("%m-%d\n%H:%M")
    ax.xaxis.set_major_formatter(date_form)
    for row,col in pixel_plot_list:
        interest_df = df.loc[(df['col']==col) & (df['row']==row)].reset_index(drop=True)
        tmp_df = interest_df.loc[interest_df['chip_name']==iboard]
        tmp_df = tmp_df.loc[tmp_df['timestamp'] > cut_time]
        ax.plot(tmp_df['timestamp'], tmp_df['baseline'], label=f'Pixel R{row} C{col}', marker='.', c='r' if col<8 else 'b')

    if draw_dates:
        for index, row in run_time_df.iterrows():
            ax.axvline(x=row['Start_Time'], color='magenta', linestyle='dashed')
            ax.axvline(x=row['End_Time'], color='black', linestyle='dashdot')

## Temperature

In [ ]:
con = sqlite3.connect('/media/daq/X9/DESYJune2024/ETROC-History/ADCHistory.sqlite')
adc_df = pd.read_sql_query("SELECT * from adc", con)

In [ ]:
adc_df['timestamp'] = pd.to_datetime(adc_df['timestamp'], format='%Y-%m-%d %H:%M:%S.%f')
adc_df['timestamp'] = adc_df['timestamp'].dt.strftime('%Y-%m-%d %H:%M:%S')
adc_df['timestamp'] = pd.to_datetime(adc_df['timestamp'])

In [ ]:
names = ['BAR 4', 'BAR 12', 'BAR 13', 'BB 1-3', 'Ambient Temp']

fig, ax = plt.subplots(figsize=(20, 10))
hep.cms.text(loc=0, ax=ax, text="Phase-2 Preliminary", fontsize=25)
ax.set_title('Temperature', size=17, loc="right")
date_form = mdates.DateFormatter("%m-%d")
ax.xaxis.set_major_formatter(date_form)

for idx, iboard in enumerate(sorted(adc_df['channel'].unique())):
    tmp_df = adc_df.loc[adc_df['channel']==iboard]
    ax.plot(tmp_df['timestamp'], tmp_df['voltage'], label=f'{names[idx]}')

text_y = [0.51, 0.55]
for index, row in run_time_df.iterrows():

    if index%2 == 0:
        yval = text_y[0]
    else:
        yval = text_y[1]

    plt.axvline(x=row['Start_Time'], color='magenta', linestyle='dashed')
    ax.text(x=(row['Start_Time'] - pd.Timedelta(hours=1.2)), y=yval, s=f"Run {row['Run']} Start", rotation=90, va='center', fontsize=12, color='magenta')
    plt.axvline(x=row['End_Time'], color='black', linestyle='dashdot')
    ax.text(x=(row['End_Time'] + pd.Timedelta(minutes=15)), y=yval, s=f"Run {row['Run']} End", rotation=90, va='center', fontsize=12)

ax.legend(loc='best')
ax.set_ylabel('Voltage')
plt.ylim(bottom=0.48, top=.6)

### VTemp per board

In [ ]:
names = ['BAR 4', 'BAR 12', 'BAR 13', 'BB 1-3', 'Ambient Temp']
draw_text_loc = [0.57, 0.534, 0.5325, .45, .499]
draw_dates = True
voltage_cut = 0.3

for idx, iboard in enumerate(sorted(adc_df['channel'].unique())):
    fig, ax = plt.subplots(figsize=(20, 10))
    hep.cms.text(loc=0, ax=ax, text="Phase-2 Preliminary", fontsize=25)
    ax.set_title(f'Board {names[idx]} Temperature', size=17, loc="right")
    tmp_df = adc_df.loc[adc_df['channel']==iboard]
    tmp_df = tmp_df.loc[tmp_df['voltage'] > voltage_cut]
    ax.plot(tmp_df['timestamp'], tmp_df['voltage'], label=f'{names[idx]}')

    date_form = mdates.DateFormatter("%m-%d\n%H:%M")
    ax.xaxis.set_major_formatter(date_form)

    if(draw_dates):
        for index, row in run_time_df.iterrows():

            plt.axvline(x=row['Start_Time'], color='magenta', linestyle='dashed')
            ax.text(x=(row['Start_Time'] - pd.Timedelta(hours=1.2)), y=draw_text_loc[idx], s=f"Run {row['Run']} Start", rotation=90, va='center', fontsize=12, color='magenta')
            plt.axvline(x=row['End_Time'], color='black', linestyle='dashdot')
            ax.text(x=(row['End_Time'] + pd.Timedelta(minutes=15)), y=draw_text_loc[idx], s=f"Run {row['Run']} End", rotation=90, va='center', fontsize=12)

    ax.legend(loc='best')
    ax.set_ylabel('Voltage')
    # ax.set_ylim(bottom=0.48, top=.6)

In [ ]:
names = ['BAR 4', 'BAR 12', 'BAR 13', 'BB 1-3', 'Ambient Temp']
draw_text_loc = [20, 0.534, 0.5325, .45, .499]
draw_dates = True

cut_time = pd.to_datetime('2024-06-04 12:30:00')

for idx, iboard in enumerate(sorted(adc_df['channel'].unique())):
    if(idx!=0): continue
    fig, ax = plt.subplots(figsize=(20, 10))
    hep.cms.text(loc=0, ax=ax, text="Phase-2 Preliminary", fontsize=25)
    ax.set_title(f'Board {names[idx]} Temperature', size=17, loc="right")
    tmp_df = adc_df.loc[adc_df['channel']==iboard]
    tmp_df = tmp_df.loc[tmp_df['timestamp'] > cut_time]
    ax.plot(tmp_df['timestamp'], (402.19*tmp_df['voltage']) - 207.81, label=f'{names[idx]}')

    date_form = mdates.DateFormatter("%m-%d\n%H:%M")
    ax.xaxis.set_major_formatter(date_form)

    if(draw_dates):
        for index, row in run_time_df.iterrows():

            plt.axvline(x=row['Start_Time'], color='magenta', linestyle='dashed')
            ax.text(x=(row['Start_Time'] - pd.Timedelta(hours=1.2)), y=draw_text_loc[idx], s=f"Run {row['Run']} Start", rotation=90, va='center', fontsize=12, color='magenta')
            plt.axvline(x=row['End_Time'], color='black', linestyle='dashdot')
            ax.text(x=(row['End_Time'] + pd.Timedelta(minutes=15)), y=draw_text_loc[idx], s=f"Run {row['Run']} End", rotation=90, va='center', fontsize=12)

    ax.legend(loc='best')
    ax.set_ylabel('Temperature [C]')
    # ax.set_ylim(bottom=0.48, top=.6)

## HV currents

In [ ]:
# watch python3 scripts/logNICrate.py -o May6 -d /media/daq/X9/DESYJune2024/ETROC-History/HV_logging -t 3

In [ ]:
HV_json = '/media/daq/X9/DESYJune2024/ETROC-History/HV_logging/JunTB.json'

In [ ]:
df = pd.read_json(HV_json, lines=True)

In [ ]:
def convert_voltage(series):
    multipliers = series.str.extract(r'(\d*\.\d+|\d+)( uV| mV| V)?', expand=True)
    multipliers[1] = multipliers[1].fillna('')
    multipliers[1] = multipliers[1].map({' uV': 1e-6, ' mV': 1e-3, ' V': 1, '': 1})
    return multipliers[0].astype('float32') * multipliers[1].astype('float32')

In [ ]:
hv_info = defaultdict(pd.DataFrame)

# Extract columns for processing
channels = df['Channel'].apply(pd.Series)
sense_voltages = df['Measured Sense Voltage'].apply(pd.Series)
terminal_voltages = df['Measured Terminal Voltage'].apply(pd.Series)
currents = df['Measured Current'].apply(pd.Series).replace(' uA', '', regex=True).astype('float32')

for inum in channels.columns:
    senseV = convert_voltage(sense_voltages[inum])
    terminalV = convert_voltage(terminal_voltages[inum])

    new_df = pd.DataFrame({
        'timestamp': pd.to_datetime(df['timestamp']),
        'senseV': -1*senseV,
        'terminalV': -1*terminalV,
        'currents': currents[inum]
    })

    hv_info[inum] = new_df

    del new_df

In [ ]:
hv_info[0]

In [ ]:
names = ['BAR 4', 'BAR 12', 'BAR 13', 'BB 1-3']

fig, ax = plt.subplots(figsize=(30, 14))
hep.cms.text(loc=0, ax=ax, text="Phase-2 Preliminary", fontsize=25)

date_form = mdates.DateFormatter("%m-%d")
ax.xaxis.set_major_formatter(date_form)

for idx, iname in enumerate(names):
    ax.plot(hv_info[idx]['timestamp'], hv_info[idx]['currents'], label=iname)

ax.legend(loc='best')
ax.set_ylabel('HV Currents [uA]')

In [ ]:
names = ['BAR 4', 'BAR 12', 'BAR 13', 'BB 1-3']

fig, ax = plt.subplots(figsize=(30, 14))
hep.cms.text(loc=0, ax=ax, text="Phase-2 Preliminary", fontsize=25)

for idx, iname in enumerate(names):
    ax.plot(hv_info[idx]['terminalV'], hv_info[idx]['currents'], label=iname)

ax.legend(loc='best')
ax.set_ylabel('HV Currents [uA]')
ax.set_xlabel('HV Setting [V]')
ax.invert_xaxis()
ax.invert_yaxis()

In [ ]:
con = sqlite3.connect('/media/daq/X9/DESYJune2024/ETROC-History/ti1080_sensor_data.sqlite')

ti_hdc1080_df = pd.read_sql_query("SELECT * from ti_hdc1080", con)
ti_hdc1080_df['timestamp'] = pd.to_datetime(ti_hdc1080_df['timestamp'], format='%Y-%m-%d %H:%M:%S.%f')
ti_hdc1080_df['timestamp'] = ti_hdc1080_df['timestamp'].dt.strftime('%Y-%m-%d %H:%M:%S')
ti_hdc1080_df['timestamp'] = pd.to_datetime(ti_hdc1080_df['timestamp'])

In [ ]:
names = ['FBK Sensor, ET2.01 (BAR 4)','HPK Sensor, ET2.01 (BAR 12)','HPK Sensor, ET2.01 (BAR 13)','HPK Split 3 Sensor, ET2.0 (BB 1-3)']
lims  = [(11.75,9.75,11.5),(.3,.1,.175),(.3,.1,.175),(.6,.1,.2)]
idxs  = [0,1,2,3]
draw_dates = False

for idx, name, lim in zip(idxs, names, lims):

    fig, ax = plt.subplots(figsize=(20, 10))
    hep.cms.text(loc=0, ax=ax, text="ETL ETROC Test Beam", fontsize=25)

    date_form = mdates.DateFormatter("%m-%d\n%H:%M")
    ax.xaxis.set_major_formatter(date_form)

    ax.plot(hv_info[idx]['timestamp'], -1*hv_info[idx]['currents'], label=iname, color='k')

    # ax.legend(loc='best')
    ax.set_ylabel('HV Currents [uA]', fontsize=25)
    ax.set_ylim(top=lim[0], bottom=lim[1])
    ax.set_xlim(left=datetime.datetime(2024, 6, 4, 19, 00), right=datetime.datetime(2024, 6, 6, 12, 0))
    ax.set_title(name, loc="right")
    if(draw_dates):
        for index, row in run_time_df.iterrows():
            if row['Start_Time'] > pd.to_datetime('2024-06-06 13:00:00'):
                continue
            plt.axvline(x=row['Start_Time'], color='magenta', linestyle='dashed')
            ax.text(x=(row['Start_Time'] - pd.Timedelta(hours=1.2)), y=lim[2], s=f"Run {row['Run']} Start", rotation=90, va='center', fontsize=12, color='magenta')
            plt.axvline(x=row['End_Time'], color='black', linestyle='dashdot')
            ax.text(x=(row['End_Time'] + pd.Timedelta(minutes=15)), y=lim[2], s=f"Run {row['Run']} End", rotation=90, va='center', fontsize=12)

    ax2 = ax.twinx()  # instantiate a second axes that shares the same x-axis
    color = 'tab:blue'
    ax2.set_ylabel('Onboard Temperature from VTemp [C]', color=color, fontsize=25)
    tmp_df = adc_df.loc[adc_df['channel']==sorted(adc_df['channel'].unique())[idx]]
    tmp_df['timestamp'] = pd.to_datetime(tmp_df['timestamp'])
    tmp_df = tmp_df.loc[tmp_df['timestamp']>pd.to_datetime('2024-06-04 17:00:00')]
    ax2.plot(tmp_df['timestamp'], (402.19*tmp_df['voltage']) - 207.81, label=f'{names[idx]}', color=color)
    ax2.tick_params(axis='y', labelcolor=color)
    ax2.spines["right"].set_color(color)
    ax2.set_ylim(20, 30)

    ax3 = ax.twinx()  # instantiate a third axes that shares the same x-axis
    ax3.spines.right.set_position(("axes", 1.07))
    color = 'tab:orange'
    ax3.set_ylabel('Ambient Temperature in Suitcase [C]', color=color, fontsize=25)
    ax3.plot(ti_hdc1080_df['timestamp'], ti_hdc1080_df['temperature'], label=f'{names[idx]}', color=color, lw=3)
    ax3.tick_params(axis='y', labelcolor=color)
    ax3.spines["right"].set_color(color)
    ax3.set_ylim(20, 30)

    ax.grid()
    plt.tight_layout()
    plt.show()

In [ ]:
names = ['BAR 4']

fig, ax = plt.subplots(figsize=(30, 14))
hep.cms.text(loc=0, ax=ax, text="Phase-2 Preliminary", fontsize=25)

date_form = mdates.DateFormatter("%m-%d\n%H:%M")
ax.xaxis.set_major_formatter(date_form)

ax.plot(hv_info[0]['timestamp'], -1*hv_info[0]['currents'], label=names[0], color='k')

ax.legend(loc='best')
ax.set_ylabel('HV Currents [uA]')
ax.set_ylim(top=12, bottom=6)
ax.set_title(name, loc="right")

if(draw_dates):
    for index, row in run_time_df.iterrows():
        plt.axvline(x=row['Start_Time'], color='magenta', linestyle='dashed')
        ax.text(x=(row['Start_Time'] - pd.Timedelta(hours=1.2)), y=9.4, s=f"Run {row['Run']} Start", rotation=90, va='center', fontsize=12, color='magenta')
        plt.axvline(x=row['End_Time'], color='black', linestyle='dashdot')
        ax.text(x=(row['End_Time'] + pd.Timedelta(minutes=15)), y=9.4, s=f"Run {row['Run']} End", rotation=90, va='center', fontsize=12)

In [ ]:
names = ['BAR 4', 'BAR 12', 'BAR 13', 'BB 1-3']

fig, ax = plt.subplots(figsize=(30, 14))
hep.cms.text(loc=0, ax=ax, text="Phase-2 Preliminary", fontsize=25)

date_form = mdates.DateFormatter("%m-%d")
ax.xaxis.set_major_formatter(date_form)

for idx, iname in enumerate(names):
    if(idx==0): continue
    ax.plot(hv_info[idx]['timestamp'], -1*hv_info[idx]['currents'], label=iname)

ax.legend(loc='best')
ax.set_ylabel('HV Currents [uA]')

if(draw_dates):
    for index, row in run_time_df.iterrows():
        plt.axvline(x=row['Start_Time'], color='magenta', linestyle='dashed')
        ax.text(x=(row['Start_Time'] - pd.Timedelta(hours=1.2)), y=lim[2], s=f"Run {row['Run']} Start", rotation=90, va='center', fontsize=12, color='magenta')
        plt.axvline(x=row['End_Time'], color='black', linestyle='dashdot')
        ax.text(x=(row['End_Time'] + pd.Timedelta(minutes=15)), y=lim[2], s=f"Run {row['Run']} End", rotation=90, va='center', fontsize=12)

In [ ]:
names = ['BAR 4', 'BAR 12', 'BAR 13', 'BB 1-3']

fig, ax = plt.subplots(figsize=(30, 14))
hep.cms.text(loc=0, ax=ax, text="Phase-2 Preliminary", fontsize=25)

for idx, iname in enumerate(names):
    if(idx==0):continue
    ax.plot(-1*hv_info[idx]['terminalV'], -1*hv_info[idx]['currents'],"--o", label=iname)

ax.legend(loc='best')
ax.set_ylabel('HV Currents [uA]')
ax.set_xlabel('HV Setting [V]')

In [ ]:
name = 'FBK UFSD4 W15 2-3 with ETROC2.01 Chip (BAR 4)'

cut_time = pd.to_datetime('2024-06-05 00:20:00')

bar_4_df = hv_info[0]
bar_4_df = bar_4_df.loc[bar_4_df['timestamp'] < cut_time]

fig, ax = plt.subplots(figsize=(30, 14))
hep.cms.text(loc=0, ax=ax, text="Phase-2 Preliminary", fontsize=25)

ax.plot(-1*bar_4_df['terminalV'], -1*bar_4_df['currents'], "--o", label=name)

ax.legend(loc='best')
ax.set_ylabel('HV Currents [uA]')
ax.set_xlabel('HV Setting [V]')

In [ ]:
names = ['BAR 4', 'BAR 12', 'BAR 13', 'BB 1-3']

fig, ax = plt.subplots(figsize=(30, 14))
hep.cms.text(loc=0, ax=ax, text="Phase-2 Preliminary", fontsize=25)

for idx, iname in enumerate(names):
    if(idx!=0):continue
    ax.plot(-1*hv_info[idx]['terminalV'], -1*hv_info[idx]['currents'], "--o", label=iname)

ax.legend(loc='best')
ax.set_ylabel('HV Currents [uA]')
ax.set_xlabel('HV Setting [V]')

In [ ]:
names = ['FBK Sensor, ET2.01 (BAR 4)','HPK Sensor, ET2.01 (BAR 12)','HPK Sensor, ET2.01 (BAR 13)','HPK Split 3 Sensor, ET2.0 (BB 1-3)']
lims  = [(40,0,11.5),(.3,.1,.175),(.3,.1,.175),(.6,.1,.2)]
idxs  = [0,1,2,3]

for idx, name, lim in zip(idxs, names, lims):

    fig, ax = plt.subplots(figsize=(30, 14))
    hep.cms.text(loc=0, ax=ax, text="Phase-2 Preliminary", fontsize=25)

    date_form = mdates.DateFormatter("%m-%d\n%H:%M")
    ax.xaxis.set_major_formatter(date_form)

    ax.plot(hv_info[idx]['timestamp'], -1*hv_info[idx]['currents'], label=iname, color='k', lw=2)

    # ax.legend(loc='best')
    ax.set_ylabel('HV Currents [uA]', fontsize=25)
    ax.set_ylim(top=lim[0], bottom=lim[1])
    ax.set_title(name, loc="right")

    if(draw_dates):
        for index, row in run_time_df.iterrows():
            plt.axvline(x=row['Start_Time'], color='magenta', linestyle='dashed')
            ax.text(x=(row['Start_Time'] - pd.Timedelta(hours=1.2)), y=lim[2], s=f"Run {row['Run']} Start", rotation=90, va='center', fontsize=12, color='magenta')
            plt.axvline(x=row['End_Time'], color='black', linestyle='dashdot')
            ax.text(x=(row['End_Time'] + pd.Timedelta(minutes=15)), y=lim[2], s=f"Run {row['Run']} End", rotation=90, va='center', fontsize=12)

    ax2 = ax.twinx()  # instantiate a third axes that shares the same x-axis
    color = 'tab:blue'
    ax2.set_ylabel('HV Setting [V]', color=color, fontsize=25)
    ax2.plot(hv_info[idx]['timestamp'], -1*hv_info[idx]['terminalV'], color=color, lw=3)
    ax2.tick_params(axis='y', labelcolor=color)
    ax2.spines["right"].set_color(color)
    ax2.set_ylim(100, 300)

    ax3 = ax.twinx()  # instantiate a third axes that shares the same x-axis
    ax3.spines.right.set_position(("axes", 1.05))
    color = 'tab:orange'
    ax3.set_ylabel('Ambient Temperature in Suitcase [C]', color=color, fontsize=25)
    ax3.plot(ti_hdc1080_df['timestamp'], ti_hdc1080_df['temperature'], label=f'{names[idx]}', color=color, lw=3)
    ax3.tick_params(axis='y', labelcolor=color)
    ax3.spines["right"].set_color(color)
    ax3.set_ylim(17, 31)

    ax.grid(which="both")
    plt.tight_layout()
    plt.show()